# PY525 Fall 2023 - Activity 10

## Sparse matrix operations

In this exercise, we implement some basic algorithms for sparse matrices in order to solidify some of the material presented in the lecture.  Specifically, we look at the COO and CSR formats that were discussed in detail.

While in actual aplications it is in general highly recommened to use an established library implementation such as `scipy.sparse`, here we look at direct implementations of the basic data structures.

In order to have a definite example to work with, consider the following sample matrix with size 10x10, given in COO format as three individual lists:

In [1]:
import numpy as np
from collections import OrderedDict

In [2]:
rs = [0, 0, 2, 3, 4, 3, 9, 8]
cs = [0, 4, 1, 3, 5, 1, 7, 8]
vs = [2.0, -1.0, 3.0, 5.0, -1.0, 4.0, 2.0, -8.0]
vec = [1, 2, 3,4,5,6,7,8,9,10]
shape=(10,10)

*(a)* Write a class that wraps a COO sparse matrix in the format given above.  It should store the data internally as `numpy` arrays.

In [3]:
class COOMatrix:
    def __init__(self, shape, rs, cs, vs, arr=[[]]):
      self._shape = shape
      self._rs = np.array(rs)
      self._cs = np.array(cs)
      self._vs = np.array(vs)
      self._arr = np.array(arr)

   # --- shape -----------------------------------------------
    @property
    def shape(self):
        """Get the 'shape' property."""
        return self._shape

    @shape.setter
    def shape(self, value):
        self._shape = value

    @shape.deleter
    def shape(self):
        del self._shape
   # --- rs -----------------------------------------------
    @property
    def rs(self):
        """Get the 'rs' property."""
        return self._rs

    @rs.setter
    def rs(self, value):
        self._rs = np.array(value)

    @rs.deleter
    def rs(self):
        del self._rs
# --- cs -----------------------------------------------
    @property
    def cs(self):
        """Get the 'cs' property."""
        return self._cs

    @cs.setter
    def cs(self, value):
        self._cs = np.array(value)

    @cs.deleter
    def cs(self):
        del self._cs
# --- vs -----------------------------------------------
    @property
    def vs(self):
        """Get the 'vs' property."""
        return self._vs

    @vs.setter
    def vs(self, value):
        self._vs = np.array(value)

    @vs.deleter
    def vs(self):
        del self._vs

    def extend(self, arr, size):
      """ extend arr to length 'size'"""
      if not isinstance(arr, np.ndarray): arr = np.array(arr)
      add_cnt = size - len(arr)
      some_zeros = np.zeros(add_cnt)
      new_arr = np.append(arr, some_zeros)
      return new_arr

    def array(self):
      self._arr = np.zeros(self._shape)
      samps = len(self._vs)
      for i in range(samps):
        r = self._rs[i]
        c = self._cs[i]
        self._arr[r, c] = self._vs[i]
        # print(f"arr[{r},{c}] = {self._vs[i]}")
      return self._arr

# --- arr -----------------------------------------------
    @property
    def arr(self):
        """Get the 'arr' property."""
        return self._arr

    @arr.setter
    def arr(self, value):
        self._arr = np.array(value)

    @arr.deleter
    def arr(self):
        del self._arr

    def Wrap(self):
      COO_Matrix = ([[self.rs], [self.cs], [self.vs]])
      return COO_Matrix

    def apply(self, _vec):
      vec = self.extend(_vec, self._shape[1])
      res = np.zeros(np.shape(vec))
      # print("rs: ", rs)
      # print("cs: ", cs)
      # print("vs: ", vs)
      # print("vec:", vec)
      # print(f"res: {res}")
      # print("-"*50)
      samps = len(vs)

      for ii, r in enumerate(rs):
        c = cs[ii]
        # print(f"({r},{c})")
        res[r] += vec[c]* vs[ii]
        # print(f"res[{r}] += vec[{r}]* vs[{c}]: = {vec[c]} * {vs[ii]}")
        # print(f"res: {res}")
        # print("-"*50)
      return res


In [8]:

coo = COOMatrix(shape, rs, cs, vs)
coo.array()
# print("arr")
# print(coo.arr)
# print("="*50)
vec = [1,2,3,4,5,6,7,8,9,10]
ans = coo.apply(vec)
print("result: ", ans)

result:  [ -3.   0.   6.  28.  -6.   0.   0.   0. -72.  16.]


*(b)* Add to the class from part (a) a method `apply` that applies the matrix to a vector and returns a new vector.

In [9]:
# See above

*(c)* Now write another class that wraps a sparse matrix in CSR format.  It should feature two constructors, one that accepts a `COOMatrix`, and one that accepts the bare COO data.  *In particular, manually implement the COO to CSR conversion.*

In [19]:
class CSRMatrix():
  """
  Holds a sparse matrix in CSR format. Can either be constructed from a COO matrix or by passing the arrays directly.
  """
  def __init__(self, shape=None, rs=None, cs=None, vs=None, coo=None):
    """ this constructor takes in the coo params, constructs a coo matrix and then converts it to csr and holds the csr params within the object created"""
    if coo is not None:
      self._construct_from_coo(coo)
    else:
      self._construct_from_params(shape, rs, cs, vs)

  def _construct_from_params(self, shape, rs, cs, vs):
    """ this constructor takes in the coo params, constructs a coo matrix and then converts it to csr and holds the csr params within the object created"""
       
    if not (len(vs) == len(rs) == len(cs)):
      raise ValueError("vs, rs, and col arrays must be of the same length.")

    if rs is not isinstance(rs, np.ndarray): rs = np.array(rs)
    if cs is not isinstance(cs, np.ndarray): cs = np.array(cs)
    if vs is not isinstance(vs, np.ndarray): vs = np.array(vs)
    
    num_rows = shape[0]

    # Sort the entries by row index (since we're CSR) using argsort
    sorted_indices = np.argsort(rs)
    sorted_rs = rs[sorted_indices]
    sorted_cs = cs[sorted_indices]
    sorted_vs = vs[sorted_indices]

    values = np.array(sorted_vs)
    col_indexes = np.array(sorted_cs)
    row_pointers = np.zeros(num_rows + 1, dtype=int)

    # Use the sorted rows to fill row_pointers - magic happens here
    np.add.at(row_pointers, sorted_rs + 1, 1)
    np.cumsum(row_pointers, out=row_pointers)

    self._values = values
    self._col_indexes = col_indexes
    self._row_pointers = row_pointers
  
  def _construct_from_coo(cls, coo):
    """ this 'constructor' just takes in a coo matrix, converts it to csr and holds the csr params within the object created"""
    cls._construct_from_params(shape=coo.shape, rs=coo.rs, cs=coo.cs, vs=coo.vs)
  
  # --- values -----------------------------------------------
  @property
  def values(self):
      """Get the 'values' property."""
      return self._values

  @values.setter
  def values(self, value):
      self._values = value

  @values.deleter
  def values(self):
      del self._values
# --- col_indexes -----------------------------------------------
  @property
  def col_indexes(self):
      """Get the 'col_indexes' property."""
      return self._col_indexes

  @col_indexes.setter
  def col_indexes(self, value):
      self._col_indexes = value

  @col_indexes.deleter
  def col_indexes(self):
      del self._col_indexes
      
# --- row_pointers -----------------------------------------------
  @property
  def row_pointers(self):
      """Get the 'row_pointers' property."""
      return self._row_pointers

  @row_pointers.setter
  def row_pointers(self, value):
      self._row_pointers = value

  @row_pointers.deleter
  def row_pointers(self):
      del self._row_pointers      

  def apply(self, vector):
    """ Applies the CSR-structured matrix to the input vector. Returns the resulting vector."""
    if len(vector) != self.row_pointers.size - 1:
        raise ValueError("Vector size must match the number of columns of the matrix.")
    
    if not isinstance(vector, np.ndarray): vector = np.array(vector)

    result = np.zeros(self.row_pointers.size - 1, dtype=vector.dtype)
    for i in range(len(result)):
        start_index = self.row_pointers[i]
        end_index = self.row_pointers[i + 1]
        for j in range(start_index, end_index):
            result[i] += self.values[j] * vector[self.col_indexes[j]]
    return result



In [24]:
# some testing::

coo = COOMatrix(shape, rs, cs, vs)
vec = [1,2,3,4,5,6,7,8,9,10]

csr1 = CSRMatrix(coo=coo)
print(f"""csr1: 
      vs: {csr1.values}
      ci: {csr1.col_indexes}
      rp: {csr1.row_pointers}
""")
res1 = csr1.apply(vec)
print("result #1: ", res1)

print(f"shape: {shape}")
print(f"rs: {rs}")

csr2 = CSRMatrix(shape=shape, rs=rs, cs=cs, vs=vs)
print(f"""csr2: 
      vs: {csr1.values}
      ci: {csr1.col_indexes}
      rp: {csr1.row_pointers}
""")
res2 = csr2.apply(vec)
print("result #2: ", res2)


csr1: 
      vs: [ 2. -1.  3.  5.  4. -1. -8.  2.]
      ci: [0 4 1 3 1 5 8 7]
      rp: [0 2 2 3 5 6 6 6 6 7 8]

result #1:  [ -3   0   6  28  -6   0   0   0 -72  16]
shape: (10, 10)
rs: [0, 0, 2, 3, 4, 3, 9, 8]
csr2: 
      vs: [ 2. -1.  3.  5.  4. -1. -8.  2.]
      ci: [0 4 1 3 1 5 8 7]
      rp: [0 2 2 3 5 6 6 6 6 7 8]

result #2:  [ -3   0   6  28  -6   0   0   0 -72  16]


**Hints:**
1. You really only need to implement one constructor with the heavy lifting, the other can just wrap the first one.
2. For the COO to CSR conversion, you will need to sort the COO data.  Since the data it is managed as three individual arrays, those need to be sorted consistently.  For that, look at `numpy.argsort`.

*(d)* To the CSR matrix class now also add a method `apply` that impelements matrix-vector multiplication.

*(e)* Verify your implementations, for example by checking that the vector below is an eigenvalue of the sample matrix given above with eigenvalue $-8.0$.

In [13]:
x = np.asarray([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0])
x

array([0., 0., 0., 0., 0., 0., 0., 0., 1., 0.])